<a href="https://colab.research.google.com/github/vshalisko/PythonIntro/blob/main/Unidad_3/Proyecto_3_borrador_mejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Conectar con Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Leer un archivo de datos desde Google Drive

### Prompt **Gemini**
`leer el archivo applicant_list_7.txt ubicado en la subcarpeta Colab Data en Google Drive. Para la lectura usar la biblioteca pandas. Asignar nombres "firstname, lastname, EPH, ECH, EMA, ECS, ESP, OP1, OP2, OP3" a las columnas. Imprimir primeras lineas de la tabla resultante.`

In [174]:
import pandas as pd

# Path to the file in your Google Drive
ruta_archivo = '/content/drive/MyDrive/Colab Data/applicant_list_7.txt'

# Column names to assign
nombres_solumnas = ["nombre", "apellido", "EPH", "ECH", "EMA", "ECS", "ESP", "OP1", "OP2", "OP3"]

# Read the file into a pandas DataFrame, using space as a separator
df = pd.read_csv(ruta_archivo, sep=' ', names=nombres_solumnas)

# Print the first few rows of the DataFrame
display(df.head())

,nombre,apellido,EPH,ECH,EMA,ECS,ESP,OP1,OP2,OP3
0,Jermine,Brunton,84,81,73,92,48,Physics,Engineering,Mathematics
1,Justo,Mirfin,71,77,61,60,41,Engineering,Biotech,Chemistry
2,Uzma,Naysmythe,60,94,75,71,80,Chemistry,Engineering,Mathematics
3,Koury,Wingo,71,81,81,83,59,Engineering,Biotech,Mathematics
4,Kentrell,Hillhouse,40,69,42,58,20,Mathematics,Engineering,Biotech


### Definir la lista de los departamentos y los examenes que se consideran en cada departamento

### Prompt
`Calcula el puntaje de admisión para cada aspirante en cada uno de los departamentos que eligió (OP1, OP2, OP3). Los exámenes considerados para el cálculo del puntaje de admisión dependen del departamento:
Física: EPH, EMA, ESP
Matemáticas: EMA, ECS, ESP
Ingeniería: EMA, ECS, EPH, ESP
Química: ECH, EPH, ESP
Biotecnología: EMA, ECS, ECH, ESP
El puntaje de admisión es el promedio de las calificaciones correspondientes a los exámenes relevantes.
Agrega nuevas columnas al DataFrame para cada puntaje calculado (OP1_score, OP2_score, OP3_score).
Finalmente, imprime las primeras filas del DataFrame resultante.`

In [175]:
# lista de departamentos
departmentos = ["Biotech","Chemistry","Engineering","Mathematics","Physics"]

# función para calcular valor de puntaje para admisión,
# formula según el departamento
def calculate_admission_score(row, department):

  if department == "Physics":
    exam_scores = [row["EPH"], row["EMA"], row["ESP"]]
  elif department == "Mathematics":
    exam_scores = [row["EMA"], row["ECS"], row["ESP"]]
  elif department == "Engineering":
    exam_scores = [row["EMA"], row["ECS"], row["EPH"], row["ESP"]]
  elif department == "Chemistry":
    exam_scores = [row["ECH"], row["EPH"], row["ESP"]]
  elif department == "Biotech":
    exam_scores = [row["EMA"], row["ECS"], row["ECH"], row["ESP"]]
  else:
    return None  # Para caso cuando departamento no esta en la lista

  return round(sum(exam_scores) / len(exam_scores), 2)

### Agregar las columnas con resultados de aplicación de las formulas de puntaje, según el departamento

In [177]:
df['nombre_completo'] = df['nombre'] + ' ' + df['apellido']
df['OP1_score'] = df.apply(lambda row: calculate_admission_score(row, row['OP1']), axis=1)
df['OP2_score'] = df.apply(lambda row: calculate_admission_score(row, row['OP2']), axis=1)
df['OP3_score'] = df.apply(lambda row: calculate_admission_score(row, row['OP3']), axis=1)
display(df.head())

,nombre,apellido,EPH,ECH,EMA,ECS,ESP,OP1,OP2,OP3,nombre_completo,OP1_score,OP2_score,OP3_score
0,Jermine,Brunton,84,81,73,92,48,Physics,Engineering,Mathematics,Jermine Brunton,68.33,74.25,71.00
1,Justo,Mirfin,71,77,61,60,41,Engineering,Biotech,Chemistry,Justo Mirfin,58.25,59.75,63.00
2,Uzma,Naysmythe,60,94,75,71,80,Chemistry,Engineering,Mathematics,Uzma Naysmythe,78.00,71.50,75.33
3,Koury,Wingo,71,81,81,83,59,Engineering,Biotech,Mathematics,Koury Wingo,73.50,76.00,74.33
4,Kentrell,Hillhouse,40,69,42,58,20,Mathematics,Engineering,Biotech,Kentrell Hillhouse,40.00,40.00,47.25


### Preguntar por cupo máximo

In [151]:
N = int(input("Introduce el número máximo de alumnos en un departamento: "))
print(f"Límite de admisión por departamento (N): {N}")

Introduce el número máximo de alumnos en un departamento: 20
Límite de admisión por departamento (N): 20


### Definición de las funciones

In [178]:
aspirantes_admitidos = pd.DataFrame(columns=["nombre_completo", "departmento", "puntaje", "opcion"])

def get_admitted_aspirants(df, admitidos, N, departamento, opcion):
  # Funcion para agregar aspirantes admitidos a la tabla de admitidos
  print(f"\nDepartmento: {departamento}")

  # Determinar el número de lugares disponibles
  N_admitidos = admitidos[admitidos['departmento'] == departamento].shape[0]
  N_disponibles = N - N_admitidos

  # Actuar según la disponibilidad de cupo
  if N_disponibles <= 0:
    print("Departamento ya no tiene cupo.")
  else:
    # Conformar lista de aspirantes al departamento
    asp_dep = df[(df[opcion] == departamento)]
    if asp_dep.empty:
      print("No hay aspirantes en este departamento.")
    else:
      columna_puntaje = f"{opcion}_score"

      # Ordenar por puntaje en orden descendente
      asp_dep_ord = asp_dep.sort_values(by=[columna_puntaje], ascending=[False])

      # Seleccionar solo candidatos que pueden entrar en cupo
      asp_dep_N = asp_dep_ord.head(N_disponibles)

      # Definir un fragemnto de datos por agragar a la tabla de admitidos
      asp_dep_formato = pd.DataFrame({'nombre_completo': asp_dep_N['nombre_completo'],
                                    'departmento': asp_dep_N[opcion],
                                    'puntaje': asp_dep_N[columna_puntaje],
                                    'opcion': opcion})

      print("Alumnos por admitir:")
      display(asp_dep_formato)

      # actualizar la tabla de admitidos
      if(admitidos.empty):
        admitidos = asp_dep_formato
      else:
        admitidos = pd.concat([admitidos, asp_dep_formato], ignore_index=True)

  return admitidos

def remove_admitted_aspirants(df, admitidos):
  # Función para eliminar los aspirantes admitidos de la tabla de aspirantes
  lista_admitidos = list(admitidos['nombre_completo'])
  mascara = ~df['nombre_completo'].isin(lista_admitidos)
  df_depurado = df[mascara]
  return df_depurado



### Etapa 1 de admisión (Opción 1)

In [179]:
# Trabajar con la copia de la table de aspirantes
df_copy = df.copy()

# Etapa 1 de seleccion
for departmento in departmentos:
  aspirantes_admitidos = get_admitted_aspirants(df_copy, aspirantes_admitidos, N, departmento, 'OP1')
  df_copy = remove_admitted_aspirants(df_copy, aspirantes_admitidos)

lista_de_admitidos = list(aspirantes_admitidos['nombre_completo'])
print("\nNúmero de aspirantes admitidos OP1: ", str(len(lista_de_admitidos)))


Departmento: Biotech
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
31,Tawnia Alcock,Biotech,83.25,OP1
43,Genee Mccrae,Biotech,81.75,OP1
51,Randon Bradhust,Biotech,78.75,OP1
67,Narissa Worthington,Biotech,76.25,OP1
76,Mendy Macmillan,Biotech,70.75,OP1



Departmento: Chemistry
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
47,Kennedy Barrett,Chemistry,88.00,OP1
50,Crescentia Dow,Chemistry,86.67,OP1
62,Estephanie Phelps,Chemistry,85.00,OP1
81,Ayeshia Jackman,Chemistry,84.67,OP1
75,Brittania Denny,Chemistry,82.00,OP1
2,Uzma Naysmythe,Chemistry,78.00,OP1
14,Franki Dinnis,Chemistry,78.00,OP1
79,Stacey Revill,Chemistry,75.67,OP1
9,Delta Fanny,Chemistry,72.67,OP1
74,Pearl Pullins,Chemistry,62.33,OP1



Departmento: Engineering
Alumnos por admitir:


...


Departmento: Mathematics
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
80,Mir Ashley,Mathematics,84.00,OP1
32,Quinisha Clarkson,Mathematics,83.00,OP1
56,Kennon Inverarity,Mathematics,80.33,OP1
16,Sameera Procter-Baines,Mathematics,73.33,OP1
70,Shealynn Melville,Mathematics,73.33,OP1
22,Divina Butterworth,Mathematics,73.33,OP1
97,Nastassja Trustram,Mathematics,73.33,OP1
20,Wynn Crampton,Mathematics,73.00,OP1
8,Spring Burridge,Mathematics,73.00,OP1
46,Mehul Bull,Mathematics,73.00,OP1



Departmento: Physics
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
7,Laney Braithwaite,Physics,91.33,OP1
19,Blia Sagar,Physics,88.00,OP1
71,Darrah Smyth,Physics,84.33,OP1
6,Sang Muldoon,Physics,81.00,OP1
30,Jathan Birley,Physics,81.00,OP1
18,Cornellius Turney,Physics,81.00,OP1
42,Martha Hatchard,Physics,81.00,OP1
33,Dashanna Herron,Physics,80.67,OP1
66,Giovanna Keel,Physics,79.67,OP1
12,Jaemi Hallets,Physics,77.33,OP1



Número de aspirantes admitidos OP1:  76


### Etapa 2 de admisión (Opción 2)

In [180]:
# Etapa 2 de seleccion
for departmento in departmentos:
  aspirantes_admitidos = get_admitted_aspirants(df_copy, aspirantes_admitidos, N, departmento, 'OP2')
  df_copy = remove_admitted_aspirants(df_copy, aspirantes_admitidos)

lista_de_admitidos = list(aspirantes_admitidos['nombre_completo'])
print("\nNúmero de aspirantes admitidos OP1 + OP2: ", str(len(lista_de_admitidos)))


Departmento: Biotech
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
95,Mira Riley,Biotech,80.00,OP2
3,Koury Wingo,Biotech,76.00,OP2
83,Katrine Proby,Biotech,71.75,OP2
38,Kyona Catrol,Biotech,71.50,OP2
55,Tawny Crockett,Biotech,71.50,OP2
59,Lorry Bunger,Biotech,69.00,OP2
94,Deniz Blanchard,Biotech,64.75,OP2
13,Artavious Fay,Biotech,64.75,OP2
73,Jessye Allum,Biotech,61.75,OP2
1,Justo Mirfin,Biotech,59.75,OP2



Departmento: Chemistry
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
52,Dashawn Bosley,Chemistry,66.33,OP2
61,Marti Mclaws,Chemistry,61.00,OP2



Departmento: Engineering
Departamento ya no tiene cupo.

Departmento: Mathematics
Departamento ya no tiene cupo.

Departmento: Physics
Departamento ya no tiene cupo.

Número de aspirantes admitidos OP1 + OP2:  90


### Etapa 3 de admisión (Opción 3)

In [181]:
# Etapa 3 de seleccion
for departmento in departmentos:
  aspirantes_admitidos = get_admitted_aspirants(df_copy, aspirantes_admitidos, N, departmento, 'OP3')
  df_copy = remove_admitted_aspirants(df_copy, aspirantes_admitidos)

lista_de_admitidos = list(aspirantes_admitidos['nombre_completo'])
print("\nNúmero de aspirantes admitidos OP1 + OP2 + OP3: ", str(len(lista_de_admitidos)))


Departmento: Biotech
Alumnos por admitir:


,nombre_completo,departmento,puntaje,opcion
53,Nicolasa Sumpter,Biotech,74.25,OP3
65,Cheyla Hankinson,Biotech,72.50,OP3
37,Tisheena Mckenney,Biotech,67.25,OP3



Departmento: Chemistry
No hay aspirantes en este departamento.

Departmento: Engineering
Departamento ya no tiene cupo.

Departmento: Mathematics
Departamento ya no tiene cupo.

Departmento: Physics
Departamento ya no tiene cupo.

Número de aspirantes admitidos OP1 + OP2 + OP3:  93


### Presentar los resultados de admisión

In [182]:
print("Aspirantes admitidos (por departamento):")
display(aspirantes_admitidos.sort_values(by=['departmento','puntaje'], ascending=[True,False]))

print("\nAspirantes no admitidos:")
non_admitted = df_copy['nombre_completo'].unique()
non_admitted.sort()
display(pd.DataFrame({'nombre_completo': non_admitted}))

Aspirantes admitidos (por departamento):


,nombre_completo,departmento,puntaje,opcion
0,Tawnia Alcock,Biotech,83.25,OP1
1,Genee Mccrae,Biotech,81.75,OP1
76,Mira Riley,Biotech,80.00,OP2
2,Randon Bradhust,Biotech,78.75,OP1
3,Narissa Worthington,Biotech,76.25,OP1
...,...,...,...,...
71,Tyrone Fern,Physics,74.33,OP1
72,Madiha Milligan,Physics,73.00,OP1
73,Jermine Brunton,Physics,68.33,OP1
74,Ammon Janssen,Physics,68.33,OP1



Aspirantes no admitidos:


,nombre_completo
0,Halima Brydone
1,Kentrell Hillhouse
2,Luna Pheobe
3,Marygrace Wheelton
4,Ramina Ogilvie
